In [ ]:
import numpy as np

from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy
from sb3_contrib.common.recurrent.type_aliases import RNNStates
import gym
import torch as th

In [ ]:
from active_critic.utils.gym_utils import make_policy_dict

In [ ]:
env = gym.make("CartPole-v1")

In [ ]:
model = RecurrentPPO("MlpLstmPolicy", "CartPole-v1", verbose=1)

In [ ]:
lstm_stats = model._last_lstm_states

In [ ]:
obs =  th.tensor(env.reset(),device='cuda').reshape([1,-1])

In [ ]:
model.policy.train()

In [64]:
model.policy.optimizer.zero_grad()
rnn_states_list = [RNNStates(pi=lstm_stats[0], vf=lstm_stats[1])]

In [65]:
for i in range(10):
    actions, values, log_prob, rnn_states = model.policy.forward(obs = obs, lstm_states=rnn_states_list[-1], episode_starts = th.zeros([1], device='cuda'), deterministic=False)
    rnn_states_list.append(rnn_states)

In [67]:
_, log_prob, entropy = model.policy.evaluate_actions(obs, actions, rnn_states_list[-2], episode_starts = th.zeros([1], device='cuda'))

In [70]:
prob_true_act = th.exp(log_prob).mean()
log_prob = log_prob.mean()
entropy = entropy.mean()


ent_loss = 1e-2 * entropy
neglogp = -log_prob
loss = neglogp + ent_loss

In [72]:
loss.mean().backward()

In [60]:
rnn_states_list[-1][0][0].mean().backward()

In [73]:
model.policy.lstm_actor.all_weights[0][0]._grad

tensor([[ 2.5638e-07, -2.6060e-07, -1.9403e-07, -3.2640e-08],
        [-7.6038e-08,  7.7292e-08,  5.7548e-08,  9.6806e-09],
        [ 1.1578e-06, -1.1769e-06, -8.7625e-07, -1.4740e-07],
        ...,
        [-1.2186e-06,  1.2387e-06,  9.2226e-07,  1.5514e-07],
        [-6.1863e-08,  6.2883e-08,  4.6820e-08,  7.8760e-09],
        [ 1.2254e-07, -1.2456e-07, -9.2742e-08, -1.5601e-08]], device='cuda:0')

In [ ]:
ent_weight = 1e-2

eval actions:         
obs: th.Tensor,
actions: th.Tensor,
lstm_states: RNNStates,
episode_starts: th.Tensor,

-> 
values, log_prob, distribution.entropy()

In [ ]:
_, log_prob, entropy = model.policy.evaluate_actions(obs, actions, rnn_states, episode_starts = th.ones([1], device='cuda'))
#print(f'max log: {log_prob.max()}')

prob_true_act = th.exp(log_prob).mean()
log_prob = log_prob.mean()
entropy = entropy.mean()


ent_loss = ent_weight * entropy
neglogp = -log_prob
loss = neglogp + ent_loss

In [ ]:
loss

In [ ]:
model.policy.lstm_actor.forward(obs = env.reset(), lstm_states=lstm_stats, epsidode_starts = th.zeros([1]))

In [ ]:
from imitation.algorithms.bc import BC
from active_critic.utils.gym_utils import (
    make_vec_env,
    make_dummy_vec_env,
    sample_expert_transitions_rollouts,
    make_pomdp_rollouts,
    make_dummy_vec_env_pomdp,
    get_avr_succ_rew_det
)

In [ ]:
env_tag = 'pickaplce'
seq_len = 100
n_demonstrations

env, vec_expert = make_dummy_vec_env(name=env_tag, seq_len=seq_len)
val_env, _ = make_dummy_vec_env(name=env_tag, seq_len=seq_len)

transitions, rollouts = sample_expert_transitions_rollouts(
    vec_expert.predict, val_env, n_demonstrations)